In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import calendar
import random
import seaborn as sns
import numpy as np
import itertools
import warnings
from itertools import groupby
warnings.filterwarnings("ignore")
from geopy.geocoders import Nominatim

In [2]:
conn = psycopg2.connect(
host = "shopify-merchant-dump.ccm9mnr5avgs.ap-south-1.rds.amazonaws.com",
  user = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c",
  password = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_password",
  database = "toffee_coffee_roasters_69e3f806d0674953afe3d3fb2f724c7c_db",
  port = '5432'
)

In [3]:
# query = """select o.id as order_id,
#             o.created_at::date as order_date,
#             o.customer_id as user_id,
#             o.total_line_items_price as total_price,
#             o.total_discounts as total_discounts,
#             o.total_discounts / o.total_line_items_price as discount_percent,
#             oli.product_id as product_id,
#             oli.name as product_name,
#             o.total_price as order_amount,
#             CASE
#         WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
#          ELSE 'Delivered'
#      END as order_status,
#             oli.quantity as quantity,
#             p.product_type as product_category,
#             oli.price as sp,
#             osa.address1 as delivery_address,
#             a.address1 as customer_address
#             from orders o
#             inner join order_line_item oli on oli.order_id = o.id
#             inner join products p on oli.product_id=p.id
#             inner join order_shipping_address osa on osa.order_id = o.id
#             inner join addresses a on a.customer_id = o.customer_id"""

# query = ("""
#     SELECT DISTINCT
#         o.id AS order_id,
#         o.created_at::date as order_date,
#         o.total_line_items_price as total_price,
#         o.total_discounts as total_discounts,
#         o.total_discounts / o.total_line_items_price as discount_percent,
#         o.total_price as order_amount,
#         CASE
#             WHEN o.cancelled_at IS NOT NULL THEN 'Cancelled'
#             ELSE 'Delivered'
#         END as order_status,
#         o.customer_id AS user_id,
#         oli.product_id as product_id,
#         oli.name as product_name,
#         oli.quantity as quantity,
#         oli.price as sp,
#         p.product_type as product_category,
#         osa.address1 as delivery_address
#     FROM orders o
#     INNER JOIN  order_line_item oli ON oli.order_id = o.id
#     inner join products p on oli.product_id=p.id
#     inner join order_shipping_address osa on osa.order_id = o.id
#     WHERE o.customer_id IS NOT NULL
#     AND oli.product_id IS NOT NULL
# """)


In [4]:
query1='''
    select o.id as order_id,
    o.created_at::date as order_date,
    o.customer_id as user_id,
    o.total_line_items_price as MRP,
    o.total_discounts / o.total_line_items_price as discount_percent,
    o.total_discounts as discount_amount,
    o.total_price as total_price

    from orders o
    WHERE
    cancelled_at is NULL;
'''

query2='''
    select oli.order_id as order_id,
    oli.product_id as product_id,
    oli.name as product_name,
    oli.quantity as quantity,
    oli.price as sp
    from order_line_item oli;
'''

query3='''
select osa.order_id as order_id,
osa.address1 as delivery_address
from  order_shipping_address osa'''
o=pd.read_sql(query1,conn)
r=pd.read_sql(query2,conn)
a=pd.read_sql(query3,conn)
data=pd.merge(o,r,on='order_id',how='left')
data=pd.merge(data,a,on='order_id',how='left')

In [5]:
query1 = '''select
    created_at::date as OrderDate,
    id as order_id,
    customer_id as user_id,
    CASE
        WHEN cancelled_at IS NOT NULL THEN 'Cancelled'
        ELSE 'Delivered'
    END as order_status
    from orders;'''

query2 = '''select
    name as product_name,
    order_id,
    product_id,
    quantity,
    price,
    variant_title as variant
    from order_line_item;'''

query3 = '''select
    id as product_id,
    title as product_title,
    product_type as product_category,
    handle,
    tags
    from products;'''

In [6]:
# data = pd.read_sql(query, conn)

In [7]:
data = data.drop_duplicates()
data = data.dropna()

In [8]:
data

,order_id,order_date,user_id,mrp,discount_percent,discount_amount,total_price,product_id,product_name,quantity,sp,delivery_address
0,5341764845706,2023-12-08,6.377996e+12,299.0,0.0,0.0,299.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,"Bungalow no 71, NEW MOTI BAGH, OPP \nHOTEL LEE..."
1,5341763338378,2023-12-08,6.935222e+12,199.0,0.0,0.0,224.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,"34 BB street, bhadrakali, hooghly"
2,5341750689930,2023-12-08,6.704616e+12,598.0,0.0,0.0,598.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,2,299.0,"3/4 - 80 - EWS,"
3,5341738500234,2023-12-08,6.935186e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,"21 Sneha flats, 4th Main Road, Kasturbanagar, ..."
4,5341737910410,2023-12-08,6.801344e+12,299.0,0.0,0.0,324.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,299.0,Address:3/336-1 404-1 venugopala Swamy Street ...
...,...,...,...,...,...,...,...,...,...,...,...,...
95291,5382883868810,2024-01-07,6.989049e+12,1200.0,0.4,480.0,720.0,5.321648e+12,Single Malt Whiskey Barrel Aged Coffee - Frenc...,1,1200.0,Zenith Youth Boys Hostel
95292,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,5.321622e+12,Peaberry Coffee - Medium Roast - Moka Pot / 25...,1,630.0,66A/2 GAUTAMNAGAR
95293,5382880723082,2024-01-07,6.692528e+12,1260.0,0.5,630.0,655.0,6.624082e+12,Blueberry Mocha - Speciality Blend - 250 gm / ...,1,630.0,66A/2 GAUTAMNAGAR
95294,5382825181322,2024-01-07,6.988957e+12,199.0,0.0,0.0,199.0,5.321602e+12,South Indian Traditional Filter Coffee - Dark ...,1,199.0,KALIGHAT TELEPHONE EXCHANGE 47 RAMESH MITRA ROAD


In [9]:
e = pd.read_sql(query1, conn)
f = pd.read_sql(query2, conn)
g = pd.read_sql(query3, conn)

Metrics Calculations

In [10]:
data["order_date"] = pd.to_datetime(data["order_date"])
data[data["product_id"] == np.nan]
data = data.dropna()
data['WeekCount']=data['order_date'].dt.strftime('%Y-w%U')
data=data.sort_values('order_date')
WeekList=data['WeekCount'].unique()

In [11]:
# data = data[data['order_status']=='Delivered']

In [12]:
WeekList = WeekList.tolist()
output_list = []

for i in range(len(WeekList) - 1):
    output_list.append(WeekList[i])
    current_week = int(WeekList[i].split('-w')[1])
    next_week = int(WeekList[i + 1].split('-w')[1])
    missing_weeks = next_week - current_week - 1
    for j in range(1, missing_weeks + 1):
        missing_week = current_week + j
        output_list.append(f'{WeekList[i][:5]}w{missing_week:02d}')

output_list.append(WeekList[-1])

In [13]:
orderuserlevel=data.groupby(['order_id','user_id']).agg(
          order_date=pd.NamedAgg(column='order_date', aggfunc='min'),

    ).reset_index()
orderuserlevel=orderuserlevel.sort_values('order_date')
orderuserlevel['TrxnRank'] = orderuserlevel.groupby('user_id')['order_date'].rank(method='first')
orderuserlevel


data=pd.merge(data,
         orderuserlevel[['order_id','TrxnRank']],
         on="order_id",
         how ='left')
data['New']=0
data['Repeat']=0
data.loc[data['TrxnRank']==1,'New']=1
data.loc[data['TrxnRank']>1,'Repeat']=1

#AOV Contribution
data['ordervalue']=data['quantity']*data['sp']
data['AOVContribution']=data['ordervalue']/data["order_id"].nunique()
data['DeltaDaysinOrders']=(data.groupby('user_id').order_date.shift() - data.order_date).dt.days.abs()
data['DeltaOrderValues']=(data.groupby('user_id').ordervalue.shift() - data.ordervalue)
data['discount_ratio'] = data['discount_amount']/data['mrp'] #change
data['new_ordervalue']= data['ordervalue']-(data['ordervalue']*data['discount_ratio'])# change

In [14]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New Users', 'Churned Users', 'Recent & Loyal Users', 'Champion Users', 'Repeat Churned Users']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero Products', 'High Value & Slow Moving Products', 'Low Value & Slow Moving Products', 'New Business Products', 'High Repeat Products']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
avg_delta_days = data["DeltaDaysinOrders"].mean()
lst = []

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()
    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
        if ((df['Frequency']==1) & (df['r_score']>2)):
            return "New Users"
        elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
            return 'Churned Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
            return 'Recent & Loyal Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] >=2) & (df['Recency'] <= avg_delta_days)):
            return 'Champion Users'
        else:
            return 'Repeat Churned Users'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_id', 'product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
            Total_Order_Value=pd.NamedAgg(column='new_ordervalue', aggfunc='sum'),
            Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique')
    ).reset_index()
    data_new=x3[x3['TrxnRank']==1]
    xuser_count = data_new.groupby(["product_name"]).agg(
    x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique'),
    x_user_list = pd.NamedAgg(column = "user_id", aggfunc = lambda x: [x.unique()])
    ).reset_index()
    data_repeat=x3[x3['TrxnRank']==2]
    repeat_users=data_repeat['user_id'].to_list()
    l=[]
    for i in range (len(xuser_count)):
        a=(xuser_count.iloc[i,2])[0]
        b=[x for x in a if x in repeat_users]
        l.append(len(b))

    xuser_count['repeat_pool']=l

    yuser_count = data_repeat.groupby('product_name').agg(
    y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()
    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on='product_name', how='left')
    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)
    result = main_grouped_for_title.copy()

    result['Conversion_Rate_x_to_y'] = result['repeat_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)
    conversion_result = result
    productlevel=pd.merge(product_level,conversion_result,on='product_name', how='left')


    productlevel['NewRepeatConversionRatio']=productlevel['Conversion_Rate_x_to_y']
    productlevel['NewRepeatConversionRatio']=productlevel['NewRepeatConversionRatio'].fillna(0)
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels))#.astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    # productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    # productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero Products'
        elif df['NewRepeatConversion_score'] >2:
            return 'High Repeat Products'
        elif (df['CP_score'] >2):
            return 'New Business Products'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving Products'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving Products'
        else:
            return 'Low Value & Slow Moving Products'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x6 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x6, ignore_index=True)
    final_stats = pd.concat([final_stats, x6], ignore_index=True)

  except:
    lst.append(x)

In [15]:
lst

['2020-w36', '2020-w37']

In [16]:
data = data[~(data['WeekCount'].isin(lst))]
output_list = [item for item in output_list if item not in lst]

In [17]:
Weekly_Stats = {}
productsegstats_weekly = {}
user_cohort_labels = ['New Users', 'Churned Users', 'Recent & Loyal Users', 'Champion Users', 'Repeat Churned Users']
user_cohort_var_list = ['Users']#, 'Orders', 'Recency', 'AOVContribution', 'OrderValue','MeanDaysDelta', 'MeanOVDelta', 'Maturity', 'Frequency', 'AOV'
product_cohort_labels=['Hero Products', 'High Value & Slow Moving Products', 'Low Value & Slow Moving Products', 'New Business Products', 'High Repeat Products']
product_cohort_var_list=['ProductCount'] #'Orders','Users','AverageSellingPrice','QuantityperCart','CartPenetration','NewUserRatio','NewOrders','ASPXQpC','AOVContribution','ProductCountMix','OrdersMix','UsersMix','AOVContribution(%)','OrderPerSKU'
final_stats = pd.DataFrame()
avg_delta_days = data["DeltaDaysinOrders"].mean()

for x in output_list:
  x3 = data[data['WeekCount'] <= x]
  try:
    userweeklydata = x3.groupby("user_id").agg(
          Last_Date=pd.NamedAgg(column='order_date', aggfunc= lambda x: np.min(np.sort(x)[-2:])),
          First_Date=pd.NamedAgg(column='order_date', aggfunc='min'),
          OrdersCount=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Frequency=pd.NamedAgg(column='order_id', aggfunc='nunique'),
          Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
          #Mean_Days_Delta=pd.NamedAgg(column='DeltaDaysinOrders', aggfunc='mean'),
          #Mean_OV_Delta=pd.NamedAgg(column='DeltaOrderValues', aggfunc='mean')
          ).reset_index()
    globalmaxdateweek=x3['order_date'].max()
    userweeklydata['Recency']=globalmaxdateweek-userweeklydata['Last_Date']
    userweeklydata['Recency']=userweeklydata['Recency'].dt.days.abs()
    userweeklydata['MaturityAge']=np.nan
    userweeklydata.loc[userweeklydata['Frequency']>1,"MaturityAge"]=(userweeklydata['Last_Date']-userweeklydata['First_Date']).dt.days.abs()
    #userweeklydata['Mean_Days_Delta']=userweeklydata['Mean_Days_Delta'].fillna(0)
    #userweeklydata['Mean_OV_Delta']=userweeklydata['Mean_OV_Delta'].fillna(0)
    userweeklydata['Customer_AOV']=userweeklydata['Order_Value']/userweeklydata['OrdersCount']

    r_labels, f_labels,AOV_labels, m_labels,DelAOV_labels = range(4, 0, -1), range(1,5), range(1,5),range(1,5),range(1,5)
    #userweeklydata['value_score'] = userweeklydata['Order_Value'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = f_labels)).astype('int64')
    userweeklydata['r_score'] = userweeklydata['Recency'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = r_labels)).astype('int64')
   # userweeklydata['AOV_score'] = userweeklydata['Customer_AOV'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = AOV_labels)).astype('int64')
    userweeklydata['m_score']=0
    userweeklydata.loc[userweeklydata['MaturityAge'].notnull(),'m_score']= userweeklydata.loc[userweeklydata['MaturityAge'].notnull()]['MaturityAge'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = m_labels)).astype('int64')
    #userweeklydata['DelAOV'] = userweeklydata['Mean_OV_Delta'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = DelAOV_labels)).astype('int64')
    #userweeklydata['SumScore'] = userweeklydata['r_score'] + userweeklydata['m_score'] + userweeklydata['AOV_score']+ userweeklydata['DelAOV'] +userweeklydata['f_score']
    def CohortLables_3(df):
        if ((df['Frequency']==1) & (df['r_score']>2)):
            return "New Users"
        elif ((df['Frequency'] == 1) & (df['r_score'] <= 2)):
            return 'Churned Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] < 3) & (df['r_score'] >2)):
            return 'Recent & Loyal Users'
        elif ((df['Frequency'] > 1) & (df['m_score'] >=2) & (df['Recency'] <= avg_delta_days)):
            return 'Champion Users'
        else:
            return 'Repeat Churned Users'
    userweeklydata['User_Cohort_Label'] = userweeklydata.apply(CohortLables_3, axis=1)
    x3=pd.merge(x3,
         userweeklydata[['user_id','User_Cohort_Label']],
         on="user_id",
         how ='left')

    totalcarts=x3['order_id'].nunique()
    product_level=x3.groupby(['product_id', 'product_name']).agg(
            Carts=pd.NamedAgg(column='order_id', aggfunc='nunique'),
            Quantity=pd.NamedAgg(column='quantity', aggfunc='sum'),
            ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
            NewOrders=pd.NamedAgg(column='New', aggfunc='sum'),
            RepeatOrders=pd.NamedAgg(column='Repeat', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
            AOVContribution=pd.NamedAgg(column='AOVContribution', aggfunc='sum'),
            Total_Order_Value=pd.NamedAgg(column='new_ordervalue', aggfunc='sum'),
            Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique')
    ).reset_index()
    data_new=x3[x3['TrxnRank']==1]
    xuser_count = data_new.groupby(["product_name"]).agg(
    x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique'),
    x_user_list = pd.NamedAgg(column = "user_id", aggfunc = lambda x: [x.unique()])
    ).reset_index()
    data_repeat=x3[x3['TrxnRank']==2]
    repeat_users=data_repeat['user_id'].to_list()
    l=[]
    for i in range (len(xuser_count)):
        a=(xuser_count.iloc[i,2])[0]
        b=[x for x in a if x in repeat_users]
        l.append(len(b))

    xuser_count['repeat_pool']=l

    yuser_count = data_repeat.groupby('product_name').agg(
    y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')
    ).reset_index()
    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on='product_name', how='left')
    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)
    result = main_grouped_for_title.copy()

    result['Conversion_Rate_x_to_y'] = result['repeat_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)
    conversion_result = result
    productlevel=pd.merge(product_level,conversion_result,on='product_name', how='left')


    productlevel['NewRepeatConversionRatio']=productlevel['Conversion_Rate_x_to_y']
    productlevel['NewRepeatConversionRatio']=productlevel['NewRepeatConversionRatio'].fillna(0)
    productlevel=productlevel.sort_values('NewRepeatConversionRatio')
    productlevel['CartPenetration']=productlevel['Carts']/totalcarts
    productlevel['QpC']=productlevel['Quantity']/productlevel['Carts']
    productlevel['ASPXQpC']=productlevel['ASP']*productlevel['QpC']

    CP_labels, NewRatio_labels,ASP_labels, QpC_labels =  range(1,5), range(1,5),range(1,5),range(1,5)
    productlevel['CP_score'] = productlevel['CartPenetration'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = CP_labels)).astype('int64')
    productlevel['NewRepeatConversion_score'] = productlevel['NewRepeatConversionRatio'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = NewRatio_labels))#.astype('int64')
    productlevel['ASP_score'] = productlevel['ASP'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = ASP_labels)).astype('int64')
    # productlevel['QpC_score'] = productlevel['QpC'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.25, 0.5, 0.75, 1], labels = QpC_labels)).astype('int64')
    # productlevel['SumScore'] = productlevel['CP_score'] + productlevel['NewRepeatConversion_score'] + productlevel['ASP_score'] + productlevel['QpC_score']
    def ProductCohortLables2(df):
        if ((df['CP_score']>2) and (df['NewRepeatConversion_score']>2)):
            return 'Hero Products'
        elif df['NewRepeatConversion_score'] >2:
            return 'High Repeat Products'
        elif (df['CP_score'] >2):
            return 'New Business Products'
        elif (df['ASP_score'] >3):
            return 'High Value & Slow Moving Products'
        elif (df['ASP_score'] <=3):
            return 'Low Value & Slow Moving Products'
        else:
            return 'Low Value & Slow Moving Products'
    productlevel['Product Cohort Lables'] = productlevel.apply(ProductCohortLables2, axis=1)

    x3=pd.merge(x3,
         productlevel[['product_name','Product Cohort Lables']],
         on="product_name",
         how ='left')


    User_Product_Level=x3.groupby(['User_Cohort_Label','Product Cohort Lables']).agg(
        Order_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
            Users=pd.NamedAgg(column='user_id', aggfunc='nunique'),
        Order_Count=pd.NamedAgg(column='order_id', aggfunc='nunique'),

    ).reset_index()

    Weekly_Stats[x] = User_Product_Level
    x4 = data[data['WeekCount']==x]
    x5 = pd.merge(x4,userweeklydata[['user_id','User_Cohort_Label']], on='user_id', how='left')
    x6 = pd.merge(x5,productlevel[['product_name','Product Cohort Lables']], on='product_name', how='left')
    # final_stats = final_stats.append(x6, ignore_index=True)
    final_stats = pd.concat([final_stats, x6], ignore_index=True)

  except:
    pass

In [18]:
last_tag_df = productlevel[["product_name" , "Product Cohort Lables"]]

In [19]:
def group_consecutive(lst):
    result = []
    current_group = []

    for key, group in groupby(lst):
        key = next(group)

        if current_group and current_group[-1] == key:
            continue

        current_group.append(key)

    result.extend(current_group)
    return result

product_profile_df = final_stats.groupby(['product_name', 'product_id']).agg(
    Total_Value=pd.NamedAgg(column='ordervalue', aggfunc='sum'),
    ASP=pd.NamedAgg(column='sp', aggfunc='mean'),
    Total_Orders=pd.NamedAgg(column='order_id', aggfunc='nunique'),
    Product_Journey=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: group_consecutive(x.tolist())),
    Tag_Count=pd.NamedAgg(column='Product Cohort Lables', aggfunc=lambda x: x.nunique()),
    Asp_Count=pd.NamedAgg(column='sp', aggfunc=lambda x: x.nunique()),
    Asp_Lst=pd.NamedAgg(column='sp', aggfunc=lambda x: list(set(x.astype(str)))),
).reset_index()

product_profile_df["Cart_Penetration_Overall"] = product_profile_df["Total_Orders"] / final_stats["order_id"].nunique()


In [20]:
kbc=pd.merge(product_profile_df,last_tag_df, on ="product_name", how="left")

In [21]:
kbc.rename(columns={"Product Cohort Lables": "Product_Final_Tag"}, inplace=True)

In [22]:
kbc['Product_Journey'] = kbc.apply(lambda row: ['Low Value & Slow Moving'] + row['Product_Journey']
                                if 'Low Value & Slow Moving' not in row['Product_Journey']
                                else row['Product_Journey'], axis=1)

In [23]:
kbc.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag'],
      dtype='object')

In [24]:
kbc['Product_Journey'] = kbc.apply(lambda row: row['Product_Journey'] + [row['Product_Final_Tag']]
                              if row['Product_Journey'][-1] != row['Product_Final_Tag']
                              else row['Product_Journey'], axis=1)
kbc = kbc[['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag']]


In [25]:
kbc

,product_name,product_id,Total_Value,ASP,Total_Orders,Product_Journey,Tag_Count,Asp_Count,Asp_Lst,Cart_Penetration_Overall,Product_Final_Tag
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,5100.0,1700.000000,3,"[Low Value & Slow Moving, High Value & Slow Mo...",2,1,[1700.0],0.000047,High Repeat Products
1,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,42900.0,1950.000000,22,"[Low Value & Slow Moving, High Value & Slow Mo...",4,1,[1950.0],0.000348,New Business Products
2,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,12100.0,2016.666667,6,"[Low Value & Slow Moving, High Value & Slow Mo...",1,2,"[2350.0, 1950.0]",0.000095,High Value & Slow Moving Products
3,Aeropress Coffee Maker,5.321696e+12,68700.0,3816.666667,18,"[Low Value & Slow Moving, High Value & Slow Mo...",3,2,"[3800.0, 4100.0]",0.000285,New Business Products
4,Aeropress Filter Paper,7.736483e+12,2080.0,520.000000,4,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[520.0],0.000063,High Repeat Products
...,...,...,...,...,...,...,...,...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,259700.0,490.000000,510,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[490.0],0.008070,New Business Products
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,150800.0,650.000000,196,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[650.0],0.003101,New Business Products
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,23400.0,1300.000000,15,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[1300.0],0.000237,New Business Products
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7.542395e+12,61850.0,1508.536585,41,"[Low Value & Slow Moving, Low Value & Slow Mov...",3,3,"[1150.0, 1550.0, 1250.0]",0.000649,New Business Products


In [26]:
product_profile_df.to_csv("product_profile_dfdf.csv", index=False)

1-2, 2-3, 3-4, Conversions

In [27]:
# product_name, product_id, variant, product_title, product_category, handle, tags
parameter_input = "product_name"

In [28]:
# e = e[e['order_status']=='Delivered']
e.drop('order_status',axis=1,inplace=True)

In [29]:
final_stats.to_csv("final_stats_toffee.csv")

In [30]:
e_sorted = e.sort_values(by=['user_id', 'orderdate', 'order_id'])
# e['Transaction_Rank'] = e.groupby(['user_id'])['orderdate'].rank(method='first')

In [31]:
merged_df = pd.merge(f, e, on='order_id')
merged_df.dropna(inplace = True)
merged_df['Transaction_Rank'] = merged_df.groupby(['user_id',parameter_input])['orderdate'].rank(method='first')

In [32]:
final = pd.merge(merged_df, g, on='product_id')

In [33]:
final['Total Order Value'] = final['quantity'] * final['price']

In [34]:
def main(final, transactionx, transactiony, parameter):

    final1 = final.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    if parameter == "tags":
        final_tags = final1.copy()
        final_tags['tags'] = final_tags['tags'].str.split(',')
        final_tags = final_tags.explode('tags', ignore_index=True)
        final1 = final_tags
    else:
        pass

    #-------------------------------------------------------------------------------------------------------------------------------

    trans_x = final1[final1['Transaction_Rank']==transactionx]
    trans_y = final1[final1['Transaction_Rank']==transactiony]

    #-------------------------------------------------------------------------------------------------------------------------------

    xuser_count = trans_x.groupby(parameter).agg(
        x_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()

    yuser_count = trans_y.groupby(parameter).agg(
        y_trxn_pool = pd.NamedAgg(column = "user_id", aggfunc = 'nunique')).reset_index()


    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title = pd.merge(xuser_count, yuser_count, on=parameter, how='left')

    #-------------------------------------------------------------------------------------------------------------------------------

    main_grouped_for_title['y_trxn_pool'].fillna(0, inplace=True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result = main_grouped_for_title.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Conversion_Rate_x_to_y'] = result['y_trxn_pool']/result['x_trxn_pool']
    result['Conversion_Rate_x_to_y'].fillna(0, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    result['Cart_Penetration_for_x_transaction'] = result['x_trxn_pool']/(result['x_trxn_pool'].sum())
    result['Cart_Penetration_for_y_transaction'] = result['y_trxn_pool']/(result['y_trxn_pool'].sum())

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result.copy()
    result2 = result.copy()

    #-------------------------------------------------------------------------------------------------------------------------------

    result1 = result1.sort_values(by='Conversion_Rate_x_to_y', ascending=False)
    result2 = result2.sort_values(by='Cart_Penetration_for_x_transaction', ascending=False)

    #-------------------------------------------------------------------------------------------------------------------------------

    result1['Mix']= result1['Conversion_Rate_x_to_y']*100/result1['Conversion_Rate_x_to_y'].sum()
    result1['CumuSum']= result1['Mix'].cumsum()
    result1['Conversion Rate Bin']=""
    result1.loc[result1['CumuSum']<=75,"Conversion Rate Bin"]='HighConversion'
    result1.loc[(result1['CumuSum']>75) & (result1['CumuSum']<=101),"Conversion Rate Bin"]='LowConversion'

    result2['Mix']= result2['Cart_Penetration_for_x_transaction']*100/result2['Cart_Penetration_for_x_transaction'].sum()
    result2['CumuSum']= result2['Mix'].cumsum()
    result2['Cart Penetration Bin']=""
    result2.loc[result2['CumuSum']<=75,"Cart Penetration Bin"]='HighCartPenetration'
    result2.loc[(result2['CumuSum']>75) & (result2['CumuSum']<=101),"Cart Penetration Bin"]='LowCartPenetration'

    #-------------------------------------------------------------------------------------------------------------------------------

    result2 = result2[[parameter,'Cart Penetration Bin']]
    final_merged_on_title_x_to_y = pd.merge(result1, result2, on=parameter)

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y = final_merged_on_title_x_to_y[final_merged_on_title_x_to_y['Conversion Rate Bin'] != 0]

    #-------------------------------------------------------------------------------------------------------------------------------

    final_merged_on_title_x_to_y['Bin'] = final_merged_on_title_x_to_y['Conversion Rate Bin'] + '_' + final_merged_on_title_x_to_y['Cart Penetration Bin']
    # final_merged_on_title_x_to_y.drop(['Conversion Rate Bin','Cart Penetration Bin'], axis=1, inplace= True)

    #-------------------------------------------------------------------------------------------------------------------------------

    merged_title_on_bin = final_merged_on_title_x_to_y.groupby('Bin').agg({
        parameter: 'nunique'
    }).reset_index()

    #-------------------------------------------------------------------------------------------------------------------------------

    x = final_merged_on_title_x_to_y.columns
    y = []
    for i in x:
        if "x_" in i:
            i = i.replace("x_", f"{str(transactionx)}_")
        if "_x" in i:
            i = i.replace("_x", f"_{str(transactiony)}")
        if "y_" in i:
            i = i.replace("y_", f"{str(transactiony)}_")
        if "_y" in i:
            i = i.replace("_y", f"_{str(transactiony)}")
        y.append(i)
    final_merged_on_title_x_to_y.columns = y

    #-------------------------------------------------------------------------------------------------------------------------------

    return merged_title_on_bin, final_merged_on_title_x_to_y[[parameter, f"Conversion_Rate_{transactionx}_to_{transactiony}", f"Cart_Penetration_for_{transactionx}_transaction", f"Cart_Penetration_for_{transactiony}_transaction"]]

In [35]:
bins, product_data = main(final, 1, 2, parameter_input)
for i in range (2, 4):
    bins, input = main(final, i, i+1, parameter_input)
    product_data = pd.merge(product_data, input, on = parameter_input, how = 'left', suffixes=('', '_remove'))
product_data.drop([i for i in product_data.columns if 'remove' in i], axis=1, inplace=True)

In [36]:
product_profile_final = pd.merge(kbc, product_data, on="product_name", how="left")
product_profile_final.fillna(0, inplace=True)

In [37]:
data['Discount_Tag'] = data['discount_amount'].apply(lambda x: 'without_discount' if x == 0 else 'with_discount')

In [38]:
grouped_data = data.groupby(['product_id', 'product_name', 'Discount_Tag'])
average_price = grouped_data['total_price'].mean().reset_index()
result = average_price.pivot(index=['product_id', 'product_name'], columns='Discount_Tag', values='total_price').reset_index()
result.columns = ['product_id', 'product_name', 'Average_Price_With_Discount', 'Average_Price_Without_Discount']

In [39]:
def get_discount_list(discounts):
    return discounts.tolist()

grouping = data.groupby(['product_name', 'product_id']).agg(
    Discount_List=pd.NamedAgg(column='discount_amount', aggfunc=get_discount_list),
    MRP_Price=pd.NamedAgg(column='total_price', aggfunc='sum'),
    Discount_Price=pd.NamedAgg(column='discount_amount', aggfunc='sum'),
).reset_index()

In [40]:
grouping['Order_Count_Without_Discount'] = 0
grouping['Order_Count_With_Discount'] = 0
def count_positive_discounts(discount_list):
    return len([value for value in discount_list if isinstance(value, (int, float)) and value > 0.0])

def count_zero_discounts(discount_list):
    return len([value for value in discount_list if isinstance(value, (int, float)) and value == 0.0])
for index, row in grouping.iterrows():
    discount_list = row['Discount_List']
    grouping['Order_Count_With_Discount'] = grouping['Discount_List'].apply(count_positive_discounts)
    grouping['Order_Count_Without_Discount'] = grouping['Discount_List'].apply(count_zero_discounts)

In [41]:
merged_result_1 = pd.merge(product_profile_final, grouping, on=['product_id', 'product_name'], how='left')

In [42]:
merged_result_2 = pd.merge(merged_result_1, result, on=['product_id', 'product_name'], how='left')

In [43]:
merged_result_2.columns.to_list()

['product_name',
 'product_id',
 'Total_Value',
 'ASP',
 'Total_Orders',
 'Product_Journey',
 'Tag_Count',
 'Asp_Count',
 'Asp_Lst',
 'Cart_Penetration_Overall',
 'Product_Final_Tag',
 'Conversion_Rate_1_to_2',
 'Cart_Penetration_for_1_transaction',
 'Cart_Penetration_for_2_transaction',
 'Conversion_Rate_2_to_3',
 'Cart_Penetration_for_3_transaction',
 'Conversion_Rate_3_to_4',
 'Cart_Penetration_for_4_transaction',
 'Discount_List',
 'MRP_Price',
 'Discount_Price',
 'Order_Count_Without_Discount',
 'Order_Count_With_Discount',
 'Average_Price_With_Discount',
 'Average_Price_Without_Discount']

In [44]:
final_df = merged_result_2[['product_name',
 'product_id',
 'Total_Value',
 'ASP',
 'Total_Orders',
 'Product_Journey',
 'Tag_Count',
 'Asp_Count',
 'Asp_Lst',
 'Cart_Penetration_Overall',
 'Product_Final_Tag',
 'Conversion_Rate_1_to_2',
 'Cart_Penetration_for_1_transaction',
 'Cart_Penetration_for_2_transaction',
 'Conversion_Rate_2_to_3',
 'Cart_Penetration_for_3_transaction',
 'Conversion_Rate_3_to_4',
 'Cart_Penetration_for_4_transaction',
 'Discount_List',
 'Order_Count_Without_Discount',
 'Order_Count_With_Discount',
 'Average_Price_With_Discount',
 'Average_Price_Without_Discount',
 'MRP_Price',
  'Discount_Price']]

In [45]:
final_df.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag',
       'Conversion_Rate_1_to_2', 'Cart_Penetration_for_1_transaction',
       'Cart_Penetration_for_2_transaction', 'Conversion_Rate_2_to_3',
       'Cart_Penetration_for_3_transaction', 'Conversion_Rate_3_to_4',
       'Cart_Penetration_for_4_transaction', 'Discount_List',
       'Order_Count_Without_Discount', 'Order_Count_With_Discount',
       'Average_Price_With_Discount', 'Average_Price_Without_Discount',
       'MRP_Price', 'Discount_Price'],
      dtype='object')

In [46]:
product_address = data.groupby(["product_name", "product_id"]).agg(
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

In [47]:
product_address

,product_name,product_id,Delivery_Address_Counts
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,"[[39B, Shreeniwas Apartments, Manisha Society,..."
1,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,"[[550 F1, 2], [28/14, First Floor, West Patel ..."
2,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,"[[B-9/5 BPCL Apartments flat no.11, Sector 62,..."
3,Aeropress Coffee Maker,5.321696e+12,"[[Shalok Mansion, Anjaneya Temple Street, Xavi..."
4,Aeropress Filter Paper,7.736483e+12,"[[159 Sai Paradise Colony Next to Shivalaya, N..."
...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,"[[HE-1189, phase - 1, 3], [Solar House Paonana..."
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,"[[Elina bhavan #209, 6th main, JSS PUBLIC SCHO..."
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,"[[First Floor House No 5, Silver Oaks Avenue, ..."
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7.542395e+12,"[[#F2 1st Floor, 1], [806, Panchvati CHS, Panc..."


In [48]:
result_rows = []

for i in range(len(product_address)):
    product_name = product_address.at[i, 'product_name']
    add_df = product_address.at[i, 'Delivery_Address_Counts']
    max_index = max(range(len(add_df)), key=lambda x: add_df[x][1])
    result_rows.append({'product_name': product_name,
                        'address_type': 'primary',
                        'delivery_address': add_df[max_index][0],
                        'count': add_df[max_index][1]})
    result_rows.extend({'product_name': product_name,
                        'address_type': 'gifting',
                        'delivery_address': element,
                        'count': count} for j, (element, count) in enumerate(add_df) if j != max_index)

result_df = pd.DataFrame(result_rows, columns=['product_name', 'address_type', 'delivery_address', 'count'])

In [49]:
result_df

,product_name,address_type,delivery_address,count
0,Aerolatte Steam Free Milk Frother To-Go with S...,primary,"39B, Shreeniwas Apartments, Manisha Society,, ...",1
1,Aerolatte Steam Free Milk Frother To-Go with S...,gifting,"95 A Maimoon 2nd floor, Flat 202, Opposite Tat...",1
2,Aerolatte Steam Free Milk Frother To-Go with S...,gifting,"Ashok puram, Lane -1",1
3,Aerolatte Steam Free Milk Frother With Stand -...,primary,550 F1,2
4,Aerolatte Steam Free Milk Frother With Stand -...,gifting,"28/14, First Floor, West Patel Nagar",1
...,...,...,...,...
77309,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"Samarth Angan , 1315 building 2B, Off K.L.Wala...",1
77310,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,C-1301 Oberoi Gardens,1
77311,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"South chakpara, liluah, howrah- 711204 ( near ...",1
77312,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"Uplabdhi 5, Shantiniketan society Tithal road,...",1


In [50]:
result_df_tag = result_df[['product_name', 'address_type', 'delivery_address']]
result_df_tag

,product_name,address_type,delivery_address
0,Aerolatte Steam Free Milk Frother To-Go with S...,primary,"39B, Shreeniwas Apartments, Manisha Society,, ..."
1,Aerolatte Steam Free Milk Frother To-Go with S...,gifting,"95 A Maimoon 2nd floor, Flat 202, Opposite Tat..."
2,Aerolatte Steam Free Milk Frother To-Go with S...,gifting,"Ashok puram, Lane -1"
3,Aerolatte Steam Free Milk Frother With Stand -...,primary,550 F1
4,Aerolatte Steam Free Milk Frother With Stand -...,gifting,"28/14, First Floor, West Patel Nagar"
...,...,...,...
77309,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"Samarth Angan , 1315 building 2B, Off K.L.Wala..."
77310,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,C-1301 Oberoi Gardens
77311,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"South chakpara, liluah, howrah- 711204 ( near ..."
77312,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,gifting,"Uplabdhi 5, Shantiniketan society Tithal road,..."


In [51]:
final_df_with_tagging = pd.merge(data, result_df_tag, on=["product_name", "delivery_address"], how="left")

In [52]:
final_df_with_tagging

,order_id,order_date,user_id,mrp,discount_percent,discount_amount,total_price,product_id,product_name,quantity,...,New,Repeat,ordervalue,AOVContribution,DeltaDaysinOrders,DeltaOrderValues,discount_ratio,new_ordervalue,Discount_Tag,address_type
0,2424016666762,2020-09-20,3.563086e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,with_discount,gifting
1,2424016142474,2020-09-20,3.563085e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,with_discount,gifting
2,2423709794442,2020-09-20,3.562684e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,with_discount,gifting
3,2423748001930,2020-09-20,3.562720e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,with_discount,gifting
4,2424011423882,2020-09-20,3.563072e+12,650.0,0.300000,195.0,455.0,5.321724e+12,Easy Brew - Cold Brew Bags (Pack of 4),1,...,1,0,650.0,0.010281,NaN,NaN,0.300000,455.000000,with_discount,gifting
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83616,5382558941322,2024-01-07,5.073752e+12,1100.0,0.400000,440.0,660.0,5.320286e+12,Baba Budangiri Estate Coffee - Whole Beans / 5...,1,...,0,1,1100.0,0.017398,258.0,-330.0,0.400000,660.000000,with_discount,gifting
83617,5383258538122,2024-01-08,6.658749e+12,1260.0,0.500000,630.0,630.0,5.321622e+12,Peaberry Coffee - Medium Roast - French Press ...,1,...,0,1,630.0,0.009965,97.0,0.0,0.500000,315.000000,with_discount,gifting
83618,5383258538122,2024-01-08,6.658749e+12,1260.0,0.500000,630.0,630.0,6.624082e+12,Blueberry Mocha - Speciality Blend - 250 gm / ...,1,...,0,1,630.0,0.009965,0.0,0.0,0.500000,315.000000,with_discount,gifting
83619,5383244251274,2024-01-08,6.339903e+12,1135.0,0.484581,550.0,585.0,5.320258e+12,Thippanahalli Estate Coffee - French Press / 2...,1,...,0,1,585.0,0.009253,89.0,0.0,0.484581,301.519824,with_discount,gifting


In [53]:
data.columns

Index(['order_id', 'order_date', 'user_id', 'mrp', 'discount_percent',
       'discount_amount', 'total_price', 'product_id', 'product_name',
       'quantity', 'sp', 'delivery_address', 'WeekCount', 'TrxnRank', 'New',
       'Repeat', 'ordervalue', 'AOVContribution', 'DeltaDaysinOrders',
       'DeltaOrderValues', 'discount_ratio', 'new_ordervalue', 'Discount_Tag'],
      dtype='object')

In [54]:
def count_and_quote_unique(x):
    unique_values = pd.unique(x)
    quoted_values = list("'" + unique_values + "'")
    count = len(unique_values)
    return quoted_values, count

customer_address = data.groupby("user_id").agg(
    # Customer_Address=pd.NamedAgg(column='customer_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: count_and_quote_unique(x)),
    Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
).reset_index()

# Extract the counts and quoted values
# customer_address['Customer_Address'] = customer_address['Customer_Address'].apply(lambda x: x[0])
customer_address['Delivery_Address'] = customer_address['Delivery_Address'].apply(lambda x: x[0])

In [55]:
import pandas as pd

# Filter rows where address_type is 'primary' or 'gifting'
data_new = result_df[result_df['address_type'].isin(['primary', 'gifting'])]

# Group by user_id and address_type, sum the count column, and pivot the table
result_sum = pd.pivot_table(data_new, values='count', index='product_name', columns='address_type', aggfunc='sum', fill_value=0).reset_index()

# Rename the columns for clarity
result_sum.columns = ['product_name', 'Persoanl_Order_Count', 'Gifting_Order_Count']

# If there are any NaN values, replace them with 0
result_sum = result_sum.fillna(0)


In [56]:
result_sum

,product_name,Persoanl_Order_Count,Gifting_Order_Count
0,Aerolatte Steam Free Milk Frother To-Go with S...,2,1
1,Aerolatte Steam Free Milk Frother With Stand -...,20,2
2,Aerolatte Steam Free Milk Frother With Stand -...,5,1
3,Aeropress Coffee Maker,17,1
4,Aeropress Filter Paper,3,1
...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,507,3
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,193,3
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,14,1
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,40,1


In [57]:
final_df_final_new = pd.merge(final_df, result_sum, on="product_name", how="left")

In [58]:
final_df_final_new

,product_name,product_id,Total_Value,ASP,Total_Orders,Product_Journey,Tag_Count,Asp_Count,Asp_Lst,Cart_Penetration_Overall,...,Cart_Penetration_for_4_transaction,Discount_List,Order_Count_Without_Discount,Order_Count_With_Discount,Average_Price_With_Discount,Average_Price_Without_Discount,MRP_Price,Discount_Price,Persoanl_Order_Count,Gifting_Order_Count
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,5100.0,1700.000000,3,"[Low Value & Slow Moving, High Value & Slow Mo...",2,1,[1700.0],0.000047,...,0.000000,"[0.0, 0.0, 0.0]",3,0,NaN,1700.000000,5100.0,0.0,2,1
1,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,42900.0,1950.000000,22,"[Low Value & Slow Moving, High Value & Slow Mo...",4,1,[1950.0],0.000348,...,0.000000,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 265...",16,6,7135.833333,2222.687500,78378.0,13380.0,20,2
2,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,12100.0,2016.666667,6,"[Low Value & Slow Moving, High Value & Slow Mo...",1,2,"[2350.0, 1950.0]",0.000095,...,0.000000,"[0.0, 550.0, 1158.0, 0.0, 0.0, 0.0]",4,2,2661.000000,2056.250000,13547.0,1708.0,5,1
3,Aeropress Coffee Maker,5.321696e+12,68700.0,3816.666667,18,"[Low Value & Slow Moving, High Value & Slow Mo...",3,2,"[3800.0, 4100.0]",0.000285,...,0.000000,"[0.0, 0.0, 0.0, 0.0, 252.0, 110.0, 0.0, 1050.0...",11,7,6163.857143,4049.545455,87692.0,4311.0,17,1
4,Aeropress Filter Paper,7.736483e+12,2080.0,520.000000,4,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[520.0],0.000063,...,0.000000,"[0.0, 0.0, 0.0, 252.0]",3,1,898.000000,1111.666667,4233.0,252.0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,259700.0,490.000000,510,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[490.0],0.008070,...,0.002786,"[0.0, 490.0, 0.0, 0.0, 0.0, 0.0, 0.0, 220.0, 0...",51,459,450.061002,590.392157,236688.0,233572.0,507,3
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,150800.0,650.000000,196,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[650.0],0.003101,...,0.001393,"[325.0, 0.0, 1560.0, 650.0, 390.0, 390.0, 0.0,...",7,189,781.543915,1550.000000,158561.8,140232.2,193,3
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,23400.0,1300.000000,15,"[Low Value & Slow Moving, Low Value & Slow Mov...",2,1,[1300.0],0.000237,...,0.000000,"[780.0, 1530.0, 780.0, 1530.0, 1530.0, 1950.0,...",0,15,1622.666667,NaN,24340.0,22410.0,14,1
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7.542395e+12,61850.0,1508.536585,41,"[Low Value & Slow Moving, Low Value & Slow Mov...",3,3,"[1150.0, 1550.0, 1250.0]",0.000649,...,0.000000,"[0.0, 0.0, 1250.0, 0.0, 0.0, 1500.0, 0.0, 475....",31,10,842.500000,2089.161290,73189.0,12775.0,40,1


In [59]:
product_prdt = data.groupby(["product_name"]).agg(
    Average_Discount_Percentage = pd.NamedAgg(column='discount_percent', aggfunc="mean"),
    Average_Discount_List = pd.NamedAgg(column='discount_percent', aggfunc=lambda x: list(pd.unique(x).round(2))),
).reset_index()

In [60]:
product_prdt

,product_name,Average_Discount_Percentage,Average_Discount_List
0,Aerolatte Steam Free Milk Frother To-Go with S...,0.000000,[0.0]
1,Aerolatte Steam Free Milk Frother With Stand -...,0.054950,"[0.0, 0.1, 0.3, 0.24, 0.19, 0.28]"
2,Aerolatte Steam Free Milk Frother With Stand -...,0.078843,"[0.0, 0.17, 0.3]"
3,Aeropress Coffee Maker,0.039072,"[0.0, 0.05, 0.01, 0.18, 0.17, 0.02, 0.18, 0.1]"
4,Aeropress Filter Paper,0.054783,"[0.0, 0.22]"
...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,0.483786,"[0.0, 0.5, 0.24, 0.33, 0.26, 0.31, 0.29, 0.33,..."
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,0.468486,"[0.5, 0.0, 0.6, 0.4, 0.48, 0.49, 0.48, 0.43, 0..."
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,0.510016,"[0.6, 0.51, 0.43, 0.5, 0.49, 0.4, 0.34]"
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,0.181194,"[0.0, 0.33, 0.97, 0.19, 0.14]"


In [61]:
# customer_address = filtered_df.groupby(["user_id"]).agg(
#     Delivery_Address_Counts=pd.NamedAgg(column='delivery_address', aggfunc=lambda x: x.value_counts().reset_index()),
# ).reset_index()

In [62]:
# import pandas as pd

# # Assuming you have a DataFrame named customer_address
# # with columns 'user_id' and 'Delivery_Address_Counts'

# # Create a new DataFrame to store the results
# result_rows = []

# for i in range(len(customer_address)):
#     user_id = customer_address.at[i, 'user_id']
#     data = customer_address.at[i, 'Delivery_Address_Counts']

#     # Find the index of the maximum count
#     max_index = max(range(len(data)), key=lambda x: data[x][1])

#     # Add the primary address to the result DataFrame
#     result_rows.append({'user_id': user_id,
#                         'address_type': 'primary',
#                         'delivery_address': data[max_index][0],
#                         'count': data[max_index][1]})

#     # Add the rest of the addresses as gifting addresses to the result DataFrame
#     result_rows.extend({'user_id': user_id,
#                         'address_type': 'gifting',
#                         'delivery_address': element,
#                         'count': count} for j, (element, count) in enumerate(data) if j != max_index)

# # Create the result DataFrame from the list of rows
# result_df = pd.DataFrame(result_rows, columns=['user_id', 'address_type', 'delivery_address', 'count'])


In [63]:
# result_df_tag = result_df[['user_id', 'address_type', 'delivery_address']]
# # result_df_taga

In [64]:
# final_df_with_tagging = pd.merge(filtered_df, result_df_tag, on=["user_id", "delivery_address"], how="left")

In [65]:

# # Count the number of tags with 'primary' for each user_id
# primary_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'primary'].groupby(['product_name', 'product_id']).size().reset_index(name='primary_count')

# # Count the number of tags with 'gifting' for each user_id
# gifting_count = final_df_with_tagging[final_df_with_tagging['address_type'] == 'gifting'].groupby(['product_name', 'product_id']).size().reset_index(name='gifting_count')

# # Merge the two DataFrames on user_id
# result = pd.merge(primary_count, gifting_count, on=['product_name', 'product_id'], how='outer').fillna(0)



In [66]:
final_df_median_value_ASP = final_df['ASP'].median()
final_df['ASP_Category'] = pd.cut(final_df['ASP'],
                                                 bins=[float('-inf'), final_df_median_value_ASP, float('inf')],
                                                 labels=['Lower ASP', 'Higher ASP'])

In [67]:
final_df_median_value_cpfol = final_df['Cart_Penetration_for_1_transaction'].median()
final_df['Cart_Penetration_for_1_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_1_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cpfol, float('inf')],
                                                 labels=['Lower CP_First_Order_Level', 'Higher CP_First_Order_Level'])

In [68]:
final_df_median_value_cpsol = final_df['Cart_Penetration_for_2_transaction'].median()
final_df['Cart_Penetration_for_2_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_2_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cpsol, float('inf')],
                                                 labels=['Lower CP_Second_Order_Level', 'Higher CP_Second_Order_Level'])

In [69]:
final_df_median_value_cptol = final_df['Cart_Penetration_for_3_transaction'].median()
final_df['Cart_Penetration_for_3_transaction_Category'] = pd.cut(final_df['Cart_Penetration_for_3_transaction'],
                                                 bins=[float('-inf'), final_df_median_value_cptol, float('inf')],
                                                 labels=['Lower CP_Third_Order_Level', 'Higher CP_Third_Order_Level'])

In [70]:
final_df_median_value_cr12 = final_df['Conversion_Rate_1_to_2'].median()
final_df['Conversion_Rate_1_to_2_Category'] = pd.cut(final_df['Conversion_Rate_1_to_2'],
                                                 bins=[float('-inf'), final_df_median_value_cr12, float('inf')],
                                                 labels=['Lower CR_1_to_2', 'Higher CR_1_to_2'])

In [71]:
final_df_median_value_cr23 = final_df['Conversion_Rate_2_to_3'].median()
final_df['Conversion_Rate_2_to_3_Category'] = pd.cut(final_df['Conversion_Rate_2_to_3'],
                                                 bins=[float('-inf'), final_df_median_value_cr23, float('inf')],
                                                 labels=['Lower CR_2_to_3', 'Higher CR_2_to_3'])

In [72]:
final_df_median_value_cr34 = final_df['Conversion_Rate_3_to_4'].median()
final_df['Conversion_Rate_3_to_4_Category'] = pd.cut(final_df['Conversion_Rate_3_to_4'],
                                                 bins=[float('-inf'), final_df_median_value_cr34, float('inf')],
                                                 labels=['Lower CR_3_to_4', 'Higher CR_3_to_4'])

In [73]:
final_df_median_value_apwd = final_df['Average_Price_With_Discount'].median()
final_df['AP_Category'] = pd.cut(final_df['Average_Price_With_Discount'],
                                                 bins=[float('-inf'), final_df_median_value_apwd, float('inf')],
                                                 labels=['Lower Discount', 'Higher Discount'])

In [74]:
final_df.to_csv("final_df_new_df11.csv", index=False)

In [75]:
final_df.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag',
       'Conversion_Rate_1_to_2', 'Cart_Penetration_for_1_transaction',
       'Cart_Penetration_for_2_transaction', 'Conversion_Rate_2_to_3',
       'Cart_Penetration_for_3_transaction', 'Conversion_Rate_3_to_4',
       'Cart_Penetration_for_4_transaction', 'Discount_List',
       'Order_Count_Without_Discount', 'Order_Count_With_Discount',
       'Average_Price_With_Discount', 'Average_Price_Without_Discount',
       'MRP_Price', 'Discount_Price', 'ASP_Category',
       'Cart_Penetration_for_1_transaction_Category',
       'Cart_Penetration_for_2_transaction_Category',
       'Cart_Penetration_for_3_transaction_Category',
       'Conversion_Rate_1_to_2_Category', 'Conversion_Rate_2_to_3_Category',
       'Conversion_Rate_3_to_4_Category', 'AP_Category'],
      dtype='object')

In [76]:
def component_1(final_df):
    df = pd.DataFrame()

    df['SKU_Name'] = final_df["product_name"]
    df['SKU_ID'] = final_df['product_id']
    df['Total_Orders'] = final_df['Total_Orders']
    df['ASP'] = final_df["ASP"]
    df["Tag"] = final_df["Product_Final_Tag"]
    df["Cart_Penetration_at_First_Order_Level"] = final_df["Cart_Penetration_for_1_transaction"]
    df["Cart_Penetration_at_Second_Order_Level"] = final_df["Cart_Penetration_for_2_transaction"]
    df["Cart_Penetration_at_Third_Order_Level"] = final_df["Cart_Penetration_for_3_transaction"]
    df["Conversion_Rate_1_to_2"] = final_df["Conversion_Rate_1_to_2"]
    df["Conversion_Rate_2_to_3"] = final_df["Conversion_Rate_2_to_3"]
    df["Conversion_Rate_3_to_4"] = final_df["Conversion_Rate_3_to_4"]
    df["Percent_Orders_Without_Discount"] = final_df["Order_Count_Without_Discount"]/final_df["Total_Orders"]
    df["Average_Price_With_Discount"] = final_df["Average_Price_With_Discount"]

    return df

result = component_1(final_df)

In [77]:
def component_2(final_df_new_upload, level):
    df = pd.DataFrame()
    first_part, second_part = level.split(" ")
    if second_part == "ASP":
      filtered_df = final_df[final_df["ASP_Category"] == level]
    if second_part == "CP_First_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_1_transaction_Category"] == level]
    if second_part == "CP_Second_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_2_transaction_Category"] == level]
    if second_part == "CP_Third_Order_Level":
      filtered_df = final_df[final_df["Cart_Penetration_for_3_transaction_Category"] == level]
    if second_part == "CR_1_to_2":
      filtered_df = final_df[final_df["Conversion_Rate_1_to_2_Category"] == level]
    if second_part == "CR_2_to_3":
      filtered_df = final_df[final_df["Conversion_Rate_2_to_3_Category"] == level]
    if second_part == "CR_3_to_4":
      filtered_df = final_df[final_df["Conversion_Rate_3_to_4_Category"] == level]
    if second_part == "Discount":
      filtered_df = final_df[final_df["AP_Category"] == level]


    df['SKU_Name'] = filtered_df["product_name"]
    df['SKU_ID'] = filtered_df['product_id']
    df['Total_Orders'] = filtered_df['Total_Orders']
    df['ASP'] = filtered_df["ASP"]
    df["Tag"] = filtered_df["Product_Final_Tag"]
    df["Cart_Penetration_at_First_Order_Level"] = filtered_df["Cart_Penetration_for_1_transaction"]
    df["Cart_Penetration_at_Second_Order_Level"] = filtered_df["Cart_Penetration_for_2_transaction"]
    df["Cart_Penetration_at_Third_Order_Level"] = filtered_df["Cart_Penetration_for_3_transaction"]
    df["Conversion_Rate_1_to_2"] = filtered_df["Conversion_Rate_1_to_2"]
    df["Conversion_Rate_2_to_3"] = filtered_df["Conversion_Rate_2_to_3"]
    df["Conversion_Rate_3_to_4"] = filtered_df["Conversion_Rate_3_to_4"]
    df["Percent_Orders_Without_Discount"] = filtered_df["Order_Count_Without_Discount"]/filtered_df["Total_Orders"]
    df["Average_Price_With_Discount"] = filtered_df["Average_Price_With_Discount"]

    return df

result = component_2(final_df, "Lower ASP")

In [78]:
result

,SKU_Name,SKU_ID,Total_Orders,ASP,Tag,Cart_Penetration_at_First_Order_Level,Cart_Penetration_at_Second_Order_Level,Cart_Penetration_at_Third_Order_Level,Conversion_Rate_1_to_2,Conversion_Rate_2_to_3,Conversion_Rate_3_to_4,Percent_Orders_Without_Discount,Average_Price_With_Discount
4,Aeropress Filter Paper,7.736483e+12,4,520.000000,High Repeat Products,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,898.000000
6,Aeropress Rubber Seal,6.575328e+12,54,650.000000,New Business Products,0.000298,0.000000,0.000000,0.000000,0.000000,0.000000,0.870370,1727.428571
32,Annanas: PineApple Fermented Coffee - Aeropres...,5.411503e+12,61,619.114754,Hero Products,0.000732,0.000420,0.000572,0.055556,0.333333,1.000000,0.049180,766.079310
33,Annanas: PineApple Fermented Coffee - Chemex /...,5.411503e+12,3,630.000000,High Repeat Products,0.000041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,720.666667
34,Annanas: PineApple Fermented Coffee - Cold Bre...,5.411503e+12,56,603.017857,Hero Products,0.000610,0.001121,0.001144,0.177778,0.250000,0.500000,0.142857,1334.338542
...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,Vanilla Cold Brew - Easy Brew Cold Brew Bags (...,6.991108e+12,29,581.034483,Hero Products,0.000393,0.002663,0.000000,0.655172,0.000000,0.000000,0.068966,644.074074
905,Vanilla Cold Brew - Easy Brew Cold Brew Bags -...,6.991108e+12,740,489.054054,New Business Products,0.009480,0.008128,0.005149,0.082976,0.155172,0.333333,0.062162,464.512003
906,Vanilla Cold Brew - Easy Brew Cold Brew Bags -...,6.991108e+12,118,644.915254,Hero Products,0.001316,0.002663,0.003432,0.195876,0.315789,0.666667,0.025424,777.021739
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,510,490.000000,New Business Products,0.006564,0.003503,0.001716,0.051653,0.120000,0.666667,0.100000,450.061002


In [79]:
final_df.head(1)

,product_name,product_id,Total_Value,ASP,Total_Orders,Product_Journey,Tag_Count,Asp_Count,Asp_Lst,Cart_Penetration_Overall,...,MRP_Price,Discount_Price,ASP_Category,Cart_Penetration_for_1_transaction_Category,Cart_Penetration_for_2_transaction_Category,Cart_Penetration_for_3_transaction_Category,Conversion_Rate_1_to_2_Category,Conversion_Rate_2_to_3_Category,Conversion_Rate_3_to_4_Category,AP_Category
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,5100.0,1700.0,3,"[Low Value & Slow Moving, High Value & Slow Mo...",2,1,[1700.0],0.000047,...,5100.0,0.0,Higher ASP,Lower CP_First_Order_Level,Lower CP_Second_Order_Level,Lower CP_Third_Order_Level,Lower CR_1_to_2,Lower CR_2_to_3,Lower CR_3_to_4,NaN


In [80]:
query_region = """select
o.id as order_id,
o.created_at::date as order_date,
osa.city as order_city,
o.customer_id as user_id,
a.city as customer_city,
oli.product_id as product_id,
oli.name as product_name,
p.product_type as product_category

from

orders o
inner join addresses a on a.customer_id = o.customer_id
inner join order_line_item oli on oli.order_id = o.id
inner join order_shipping_address osa on osa.order_id = o.id
inner join products p on p.id = oli.product_id
"""

In [81]:
data_region = pd.read_sql(query_region, conn)

In [82]:
category = data_region.groupby("product_name").agg(         # Replace "product_name" with parameter_input in product profile
    Best_Selling_City = pd.NamedAgg(column = "order_city", aggfunc = lambda x: x.value_counts().reset_index().iloc[0:1, 0].tolist())).reset_index()
category

,product_name,Best_Selling_City
0,Aerolatte Steam Free Milk Frother To-Go with S...,[Mumbai]
1,Aerolatte Steam Free Milk Frother With Stand -...,[Dindigul]
2,Aerolatte Steam Free Milk Frother With Stand -...,[Gurgaon]
3,Aeropress Coffee Maker,[Bangalore]
4,Aeropress Filter Paper,[Bangalore]
...,...,...
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,[Mumbai]
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,[Mumbai]
912,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,[Mumbai]
913,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,[Bengaluru]


In [83]:
%%time
def get_region(lat, lon):
    if lat >= 20.0 and lat <= 35.0 and lon >= 72.5 and lon <= 77.5:
        return 'North'
    elif lat >= 8.4 and lat <= 20.0 and lon >= 77.5 and lon <= 88.0:
        return 'South'
    elif lat >= 20.0 and lat <= 27.5 and lon >= 77.5 and lon <= 88.0:
        return 'East'
    elif lat >= 8.4 and lat <= 27.5 and lon >= 70.0 and lon <= 77.5:
        return 'West'
    else:
        return 'Central'

def get_city_region(city):
    geolocator = Nominatim(user_agent="geo_locator")
    location = geolocator.geocode(city)
    if location:
        return get_region(location.latitude, location.longitude)
    else:
        return None

category['region'] = category['Best_Selling_City'].apply(lambda cities: get_city_region(cities[0]) if cities else None)

CPU times: user 1min 38s, sys: 1.58 s, total: 1min 39s
Wall time: 13min 41s


In [84]:
fill_na_value = category["region"].value_counts().reset_index().iloc[0, 0]

In [85]:
data_region_new = pd.merge(data_region, category, on = 'product_name', how = 'left')     # Replace "product_name" with parameter_input in product profile
data_region_new["region"] = data_region_new["region"].fillna(fill_na_value)
data_region_new

,order_id,order_date,order_city,user_id,customer_city,product_id,product_name,product_category,Best_Selling_City,region
0,5341764845706,2023-12-08,Delhi,6377995993226,South West Delhi,5321601712266,South Indian Traditional Filter Coffee - Dark ...,South Indian Filter Coffee,[Chennai],South
1,5341750689930,2023-12-08,Nagpur,6704615587978,Nagpur,5321601712266,South Indian Traditional Filter Coffee - Dark ...,South Indian Filter Coffee,[Chennai],South
2,5341737910410,2023-12-08,Hosur,6801344069770,Hosur,5321601712266,South Indian Traditional Filter Coffee - Dark ...,South Indian Filter Coffee,[Chennai],South
3,5341737910410,2023-12-08,Hosur,6801344069770,Hosur,5321601712266,South Indian Traditional Filter Coffee - Dark ...,South Indian Filter Coffee,[Chennai],South
4,5341683515530,2023-12-08,Islampur,6909103636618,Islampur,5321601712266,South Indian Traditional Filter Coffee - Dark ...,South Indian Filter Coffee,[Chennai],South
...,...,...,...,...,...,...,...,...,...,...
161003,5382880723082,2024-01-07,Delhi,6692528455818,Delhi,5321622093962,Peaberry Coffee - Medium Roast - Moka Pot / 25...,Single Origin Coffee,[Mumbai],West
161004,5382880723082,2024-01-07,Delhi,6692528455818,Delhi,5321622093962,Peaberry Coffee - Medium Roast - Moka Pot / 25...,Single Origin Coffee,[Mumbai],West
161005,5382880723082,2024-01-07,Delhi,6692528455818,Delhi,6624081969290,Blueberry Mocha - Speciality Blend - 250 gm / ...,,[Mumbai],West
161006,5382880723082,2024-01-07,Delhi,6692528455818,Delhi,6624081969290,Blueberry Mocha - Speciality Blend - 250 gm / ...,,[Mumbai],West


In [86]:
product_wise = data_region_new.groupby(["product_id", "product_name"]).agg(
    Purchase_Type = pd.NamedAgg(column = "region", aggfunc = lambda x: x.value_counts().reset_index().iloc[0, 0])).reset_index()
product_wise

,product_id,product_name,Purchase_Type
0,5320059781258,Thogarihunkal Estate Coffee - Aeropress / 250 gm,West
1,5320059781258,Thogarihunkal Estate Coffee - Chemex / 250 gm,West
2,5320059781258,Thogarihunkal Estate Coffee - Cold Brew / 250 gm,West
3,5320059781258,Thogarihunkal Estate Coffee - Cold Brew / 500 gm,South
4,5320059781258,Thogarihunkal Estate Coffee - Decoction Coffee...,South
...,...,...,...
910,7792635084938,Brazil & Ethiopia Speciality Instant Coffee (P...,Central
911,7792637018250,Colombia & Ethiopia Speciality Instant Coffee ...,Central
912,7792637018250,Colombia & Ethiopia Speciality Instant Coffee ...,West
913,7792640884874,Brazil & Colombia Speciality Instant Coffee (P...,North


In [87]:
final_df_final_new = pd.merge(final_df_final_new, product_wise, on=["product_id", "product_name"], how="left")

In [88]:
final_df_final_new.to_csv("finn132ew123.csv", index=False)

In [89]:
final_df_final_new = pd.merge(final_df_final_new, product_prdt, on=["product_name"], how="left")

In [90]:
final_df_final_new.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag',
       'Conversion_Rate_1_to_2', 'Cart_Penetration_for_1_transaction',
       'Cart_Penetration_for_2_transaction', 'Conversion_Rate_2_to_3',
       'Cart_Penetration_for_3_transaction', 'Conversion_Rate_3_to_4',
       'Cart_Penetration_for_4_transaction', 'Discount_List',
       'Order_Count_Without_Discount', 'Order_Count_With_Discount',
       'Average_Price_With_Discount', 'Average_Price_Without_Discount',
       'MRP_Price', 'Discount_Price', 'Persoanl_Order_Count',
       'Gifting_Order_Count', 'Purchase_Type', 'Average_Discount_Percentage',
       'Average_Discount_List'],
      dtype='object')

In [91]:
# final_df_final_new = pd.merge(final_df_final_new, product_prdt, on="product_name", how="left")

In [92]:
final_df_final_new.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Final_Tag',
       'Conversion_Rate_1_to_2', 'Cart_Penetration_for_1_transaction',
       'Cart_Penetration_for_2_transaction', 'Conversion_Rate_2_to_3',
       'Cart_Penetration_for_3_transaction', 'Conversion_Rate_3_to_4',
       'Cart_Penetration_for_4_transaction', 'Discount_List',
       'Order_Count_Without_Discount', 'Order_Count_With_Discount',
       'Average_Price_With_Discount', 'Average_Price_Without_Discount',
       'MRP_Price', 'Discount_Price', 'Persoanl_Order_Count',
       'Gifting_Order_Count', 'Purchase_Type', 'Average_Discount_Percentage',
       'Average_Discount_List'],
      dtype='object')

In [93]:
final_df_final_new = final_df_final_new.rename(columns={"Purchase_Type": "Basket_Type", "Product_Final_Tag" : "Product_Tag"})

In [94]:
final_df_final_new['Percent_Order_With_Discount'] = (final_df_final_new['Order_Count_With_Discount'] / (final_df_final_new['Order_Count_Without_Discount'] + final_df_final_new['Order_Count_With_Discount']))


In [95]:
final_df_final_new['Percent_Gifting_Order'] = (final_df_final_new['Gifting_Order_Count'] / (final_df_final_new['Gifting_Order_Count'] + final_df_final_new['Persoanl_Order_Count']))


In [96]:
final_df_final_new.columns

Index(['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
       'Product_Journey', 'Tag_Count', 'Asp_Count', 'Asp_Lst',
       'Cart_Penetration_Overall', 'Product_Tag', 'Conversion_Rate_1_to_2',
       'Cart_Penetration_for_1_transaction',
       'Cart_Penetration_for_2_transaction', 'Conversion_Rate_2_to_3',
       'Cart_Penetration_for_3_transaction', 'Conversion_Rate_3_to_4',
       'Cart_Penetration_for_4_transaction', 'Discount_List',
       'Order_Count_Without_Discount', 'Order_Count_With_Discount',
       'Average_Price_With_Discount', 'Average_Price_Without_Discount',
       'MRP_Price', 'Discount_Price', 'Persoanl_Order_Count',
       'Gifting_Order_Count', 'Basket_Type', 'Average_Discount_Percentage',
       'Average_Discount_List', 'Percent_Order_With_Discount',
       'Percent_Gifting_Order'],
      dtype='object')

In [97]:
final_df_final_new = final_df_final_new[['product_name', 'product_id', 'Total_Value', 'ASP', 'Total_Orders',
                                         'Product_Journey', 'Product_Tag', 'Cart_Penetration_Overall',
                                         'Cart_Penetration_for_1_transaction', 'Cart_Penetration_for_2_transaction',
                                         'Cart_Penetration_for_3_transaction', 'Cart_Penetration_for_4_transaction',
                                          'Conversion_Rate_1_to_2', 'Conversion_Rate_2_to_3', 'Conversion_Rate_3_to_4',
                                         'Percent_Order_With_Discount','Percent_Gifting_Order', 'Basket_Type',
                                         'Persoanl_Order_Count', 'Gifting_Order_Count', 'Average_Discount_Percentage',
                                         'Average_Price_With_Discount', 'Average_Price_Without_Discount', 'Persoanl_Order_Count', 'Gifting_Order_Count',
                                         'Tag_Count', 'Asp_Count', 'Asp_Lst', 'Discount_List', 'Order_Count_With_Discount',
                                         'Order_Count_Without_Discount', 'Discount_Price', 'MRP_Price']]

In [98]:
final_df_final_new

,product_name,product_id,Total_Value,ASP,Total_Orders,Product_Journey,Product_Tag,Cart_Penetration_Overall,Cart_Penetration_for_1_transaction,Cart_Penetration_for_2_transaction,...,Persoanl_Order_Count,Gifting_Order_Count,Tag_Count,Asp_Count,Asp_Lst,Discount_List,Order_Count_With_Discount,Order_Count_Without_Discount,Discount_Price,MRP_Price
0,Aerolatte Steam Free Milk Frother To-Go with S...,6.603461e+12,5100.0,1700.000000,3,"[Low Value & Slow Moving, High Value & Slow Mo...",High Repeat Products,0.000047,0.000054,0.000000,...,2,1,2,1,[1700.0],"[0.0, 0.0, 0.0]",0,3,0.0,5100.0
1,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,42900.0,1950.000000,22,"[Low Value & Slow Moving, High Value & Slow Mo...",New Business Products,0.000348,0.000298,0.000140,...,20,2,4,1,[1950.0],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 260.0, 265...",6,16,13380.0,78378.0
2,Aerolatte Steam Free Milk Frother With Stand -...,6.603460e+12,12100.0,2016.666667,6,"[Low Value & Slow Moving, High Value & Slow Mo...",High Value & Slow Moving Products,0.000095,0.000081,0.000000,...,5,1,1,2,"[2350.0, 1950.0]","[0.0, 550.0, 1158.0, 0.0, 0.0, 0.0]",2,4,1708.0,13547.0
3,Aeropress Coffee Maker,5.321696e+12,68700.0,3816.666667,18,"[Low Value & Slow Moving, High Value & Slow Mo...",New Business Products,0.000285,0.000095,0.000000,...,17,1,3,2,"[3800.0, 4100.0]","[0.0, 0.0, 0.0, 0.0, 252.0, 110.0, 0.0, 1050.0...",7,11,4311.0,87692.0
4,Aeropress Filter Paper,7.736483e+12,2080.0,520.000000,4,"[Low Value & Slow Moving, Low Value & Slow Mov...",High Repeat Products,0.000063,0.000000,0.000000,...,3,1,2,1,[520.0],"[0.0, 0.0, 0.0, 252.0]",1,3,252.0,4233.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,259700.0,490.000000,510,"[Low Value & Slow Moving, Low Value & Slow Mov...",New Business Products,0.008070,0.006564,0.003503,...,507,3,2,1,[490.0],"[0.0, 490.0, 0.0, 0.0, 0.0, 0.0, 0.0, 220.0, 0...",459,51,233572.0,236688.0
910,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,150800.0,650.000000,196,"[Low Value & Slow Moving, Low Value & Slow Mov...",New Business Products,0.003101,0.002387,0.002102,...,193,3,2,1,[650.0],"[325.0, 0.0, 1560.0, 650.0, 390.0, 390.0, 0.0,...",189,7,140232.2,158561.8
911,Vanilla Cold Brew Bags | Free Mason Jar - Pack...,7.570590e+12,23400.0,1300.000000,15,"[Low Value & Slow Moving, Low Value & Slow Mov...",New Business Products,0.000237,0.000203,0.000000,...,14,1,2,1,[1300.0],"[780.0, 1530.0, 780.0, 1530.0, 1530.0, 1950.0,...",15,0,22410.0,24340.0
912,Whiskey Barrel Aged Hot Brew Sachets & Enamel ...,7.542395e+12,61850.0,1508.536585,41,"[Low Value & Slow Moving, Low Value & Slow Mov...",New Business Products,0.000649,0.000000,0.000000,...,40,1,3,3,"[1150.0, 1550.0, 1250.0]","[0.0, 0.0, 1250.0, 0.0, 0.0, 1500.0, 0.0, 475....",10,31,12775.0,73189.0


In [99]:
final_df_final_new.to_csv("fina12345678.csv", index=False)